# **Train your YOLOv8 object detection model.**

First, fill in your **apikey** and **project name** below. Next, start the training script by clicking "Runtime -> Run all"

For more advanced options like setting the number of epochs or changing the base model, look in the documentation.

Once the training is finished, the new model is added to the trainyolo platform under the project models tab. There you can compare scores with other versions, and use this model for model-assisted labeling.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# DATASET PARAMETERS
API_KEY = "ff604887db95687b442be0f4cdb5a5e38255fe53"
PROJECT_NAME = "PlayerDetection"

# YOLO PARAMETERS
IM_SIZE = 640
N_EPOCHS = 50 # you can lower this if you have a large dataset
BATCH_SIZE = 2 # you can increase this if you have a large dataset, > 50 images, you can increase this to 4, >100 images you can increase this to 8
MODEL = 'yolov8n.pt'

In [2]:
# install yolov8
%pip install -qU ultralytics

# install trainyolo sdk
%pip install -qU trainyolo-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 902.2/902.2 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.9 MB/s eta 0:00:00


In [3]:
from trainyolo.client import Client, Project

# init client
client = Client(API_KEY)

# load project
project = Project.get_by_name(client, PROJECT_NAME)
project_location = project.pull(location='./data', format='yolov8')

Loading samples...


100%|██████████| 73/73 [00:00<00:00, 6711.33it/s]


100%|██████████| 73/73 [00:00<00:00, 2426.72it/s]


In [4]:
# Train yolo model
!yolo task=detect mode=train model=$MODEL data="{project_location}/dataset.yaml" epochs=$N_EPOCHS imgsz=$IM_SIZE batch=$BATCH_SIZE

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 6.25M/6.25M [00:00<00:00, 97.9MB/s]
Ultralytics 8.3.53 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./data/PlayerDetection/dataset.yaml, epochs=50, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False

In [5]:
# upload model
from trainyolo.utils.yolov8 import upload_yolov8_run

upload_yolov8_run(project, iou=0.45)

Reading best conf from F1_curve
Using conf=0.353, which maximizes f1 score.
adding weights: ./runs/detect/train/weights/best.pt to project ...


In [12]:
import os
weights_path = os.path.abspath("runs/detect/train/weights/best.pt")
print(weights_path)


/content/runs/detect/train/weights/best.pt


In [13]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt source=/content/drive/MyDrive/GP/PlayerDetectionyolov8/video.mp4 save=True


Ultralytics 8.3.53 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

video 1/1 (frame 1/2346) /content/drive/MyDrive/GP/PlayerDetectionyolov8/video.mp4: 384x640 4 Players, 34.8ms
video 1/1 (frame 2/2346) /content/drive/MyDrive/GP/PlayerDetectionyolov8/video.mp4: 384x640 4 Players, 6.5ms
video 1/1 (frame 3/2346) /content/drive/MyDrive/GP/PlayerDetectionyolov8/video.mp4: 384x640 4 Players, 6.1ms
video 1/1 (frame 4/2346) /content/drive/MyDrive/GP/PlayerDetectionyolov8/video.mp4: 384x640 4 Players, 6.3ms
video 1/1 (frame 5/2346) /content/drive/MyDrive/GP/PlayerDetectionyolov8/video.mp4: 384x640 4 Players, 6.1ms
video 1/1 (frame 6/2346) /content/drive/MyDrive/GP/PlayerDetectionyolov8/video.mp4: 384x640 4 Players, 6.4ms
video 1/1 (frame 7/2346) /content/drive/MyDrive/GP/PlayerDetectionyolov8/video.mp4: 384x640 4 Players, 6.1ms
video 1/1 (frame 8/2346) /content/drive/MyDrive/GP/PlayerDetectionyolov8

In [14]:
!ls runs/detect/predict


video.avi


In [15]:
!ffmpeg -i runs/detect/predict/video.avi -vcodec libx264 -acodec aac runs/detect/predict/video.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [16]:
from IPython.display import Video
Video("runs/detect/predict/video.mp4")
